<a href="https://colab.research.google.com/github/Saoudyahya/Reasoning-RAG-with-deep-seek/blob/main/reasining%26agenticRag_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.5/232.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 24.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0


In [2]:
pip install langchain_community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.9/351.9 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 106.9 MB/s eta 0:00:00


In [3]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 95.4 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8


In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [5]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.7 MB/s eta 0:00:00


In [ ]:
import os
import re
import time
import json
import requests
from typing import List, Dict, Any, Optional, Tuple
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    TextLoader,
    PDFMinerLoader,
    UnstructuredMarkdownLoader,
    CSVLoader
)
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.schema import Document


# Configuration class for the RAG system
class RAGConfig:
    def __init__(self):
        # DeepSeek API configuration (replace with your API key)
        self.deepseek_api_key = "sk-c6e6fbf20e3743dc98e51280d199b33d"
        self.deepseek_api_url = "https://api.deepseek.com/v1/chat/completions"

        # DeepSeek embedding model
        self.embedding_model_name = "deepseek-ai/deepseek-coder-7b-base"

        # Vector database configuration
        self.vector_db_path = "vector_db"

        # Document processing configuration
        self.chunk_size = 1000
        self.chunk_overlap = 200

        # Search configuration
        self.top_k = 5
        self.similarity_threshold = 0.7

        # Agent configuration
        self.max_iterations = 5
        self.thinking_steps = True


# Embedding class for document and query encoding
class DeepSeekEmbedding:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.embedding_model_name)
        self.model = AutoModel.from_pretrained(config.embedding_model_name)

    def get_embeddings(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for a list of texts."""
        embeddings = []

        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            with torch.no_grad():
                outputs = self.model(**inputs)

            # Use the mean of the last hidden state as the embedding
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy().tolist()
            embeddings.append(embedding)

        return embeddings


# Document processor for loading and chunking documents
class DocumentProcessor:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=config.chunk_size,
            chunk_overlap=config.chunk_overlap
        )

    def load_documents(self, directory_path: str) -> List[Document]:
        """Load documents from a directory."""
        # Configure loaders for different file types
        loaders = {
            "*.txt": TextLoader,
            "*.pdf": PDFMinerLoader,
            "*.md": UnstructuredMarkdownLoader,
            "*.csv": CSVLoader
        }

        documents = []

        for glob_pattern, loader_cls in loaders.items():
            try:
                loader = DirectoryLoader(
                    directory_path,
                    glob=glob_pattern,
                    loader_cls=loader_cls
                )
                docs = loader.load()
                documents.extend(docs)
            except Exception as e:
                print(f"Error loading {glob_pattern} documents: {e}")

        return documents

    def process_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks for embedding."""
        chunks = []

        for doc in documents:
            try:
                doc_chunks = self.text_splitter.split_documents([doc])
                chunks.extend(doc_chunks)
            except Exception as e:
                print(f"Error processing document {doc.metadata.get('source', 'unknown')}: {e}")

        return chunks


# Vector store for document storage and retrieval
class VectorStore:
    def __init__(self, config: RAGConfig, embedding_model: DeepSeekEmbedding):
        self.config = config
        self.embedding_model = embedding_model
        self.vector_store = None

    def create_vector_store(self, documents: List[Document]) -> None:
        """Create a vector store from documents."""
        texts = [doc.page_content for doc in documents]
        metadatas = [doc.metadata for doc in documents]

        embeddings = self.embedding_model.get_embeddings(texts)

        self.vector_store = FAISS.from_embeddings(
            text_embeddings=zip(texts, embeddings),
            embedding=self.embedding_model,
            metadatas=metadatas
        )

        # Save the vector store
        self.vector_store.save_local(self.config.vector_db_path)

    def load_vector_store(self) -> bool:
        """Load the vector store if it exists."""
        if os.path.exists(self.config.vector_db_path):
            self.vector_store = FAISS.load_local(
                self.config.vector_db_path,
                self.embedding_model
            )
            return True
        return False

    def similarity_search(self, query: str) -> List[Document]:
        """Search for similar documents to the query."""
        if not self.vector_store:
            raise ValueError("Vector store not initialized. Please create or load a vector store first.")

        query_embedding = self.embedding_model.get_embeddings([query])[0]

        results = self.vector_store.similarity_search_by_vector_with_relevance_scores(
            query_embedding,
            k=self.config.top_k
        )

        # Filter results by similarity threshold
        filtered_results = [
            doc for doc, score in results
            if score >= self.config.similarity_threshold
        ]

        return filtered_results


# DeepSeek LLM client for RAG
class DeepSeekLLM:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {config.deepseek_api_key}"
        }

    def generate(self, prompt: str, system_message: str = None) -> str:
        """Generate text using DeepSeek API."""
        messages = []

        if system_message:
            messages.append({"role": "system", "content": system_message})

        messages.append({"role": "user", "content": prompt})

        data = {
            "model": "deepseek-chat",
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 2000
        }

        try:
            response = requests.post(
                self.config.deepseek_api_url,
                headers=self.headers,
                json=data
            )
            response.raise_for_status()

            result = response.json()
            return result["choices"][0]["message"]["content"]
        except Exception as e:
            print(f"Error calling DeepSeek API: {e}")
            return f"Error generating response: {str(e)}"


# Agentic RAG system that combines all components
class AgenticRAG:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.embedding_model = DeepSeekEmbedding(config)
        self.document_processor = DocumentProcessor(config)
        self.vector_store = VectorStore(config, self.embedding_model)
        self.llm = DeepSeekLLM(config)

    def ingest_documents(self, directory_path: str) -> None:
        """Ingest documents from a directory."""
        print(f"Loading documents from {directory_path}...")
        documents = self.document_processor.load_documents(directory_path)
        print(f"Loaded {len(documents)} documents.")

        print("Processing documents...")
        chunks = self.document_processor.process_documents(documents)
        print(f"Created {len(chunks)} chunks.")

        print("Creating vector store...")
        self.vector_store.create_vector_store(chunks)
        print("Vector store created successfully.")

    def query(self, user_query: str) -> str:
        """Process a user query and generate a response."""
        # Try to load the vector store if not already loaded
        if not self.vector_store.vector_store:
            if not self.vector_store.load_vector_store():
                return "No documents have been ingested. Please ingest documents first."

        # Initial response
        print(f"Query: {user_query}")

        # Agentic thinking process
        response = self._agentic_process(user_query)

        return response

    def _agentic_process(self, user_query: str) -> str:
        """Execute the agentic process for responding to queries."""
        system_message = """You are an intelligent agent with access to a knowledge base.
        Your task is to provide accurate, relevant information based on the query and the retrieved context.
        Think step by step and analyze the retrieved information carefully before formulating your final response."""

        # Initial retrieval
        retrieved_docs = self.vector_store.similarity_search(user_query)

        if not retrieved_docs:
            # Handle the case when no relevant documents are found
            prompt = f"""Query: {user_query}

            No relevant documents were found in the knowledge base. Please provide a general response based on your knowledge.
            """
            return self.llm.generate(prompt, system_message)

        # For agentic reasoning, we'll use a multi-step process
        context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

        iteration_responses = []
        current_query = user_query

        for iteration in range(self.config.max_iterations):
            # Check if we should continue
            if iteration > 0 and not self._should_continue(current_query, iteration_responses[-1]):
                break

            # Generate thinking steps if enabled
            thinking = ""
            if self.config.thinking_steps:
                thinking_prompt = f"""Query: {current_query}

                Context:
                {context}

                Think step by step about this query. What are the key points to address? What information from the context is most relevant? What additional information might be needed?
                """
                thinking = self.llm.generate(thinking_prompt, system_message)

            # Generate the response
            response_prompt = f"""Query: {current_query}

            Context:
            {context}

            {thinking if thinking else ""}

            Based on the context provided, please answer the query. If the context doesn't contain enough information, acknowledge this and provide the best answer you can.
            """

            response = self.llm.generate(response_prompt, system_message)
            iteration_responses.append(response)

            # Generate follow-up questions or refinements
            refinement_prompt = f"""Query: {current_query}

            Your current response:
            {response}

            Are there aspects of the query that haven't been fully addressed? What follow-up questions would help provide a more complete answer? How could the search be refined?
            """

            refinement = self.llm.generate(refinement_prompt, system_message)

            # Extract a new query for the next iteration
            new_query_prompt = f"""Original query: {user_query}

            Current response:
            {response}

            Refinement thoughts:
            {refinement}

            Based on the above, formulate a new search query that would help address any gaps in the current response. Return ONLY the new query without any explanation.
            If you believe the query has been fully addressed, return "COMPLETE".
            """

            new_query = self.llm.generate(new_query_prompt, system_message).strip()

            if new_query == "COMPLETE" or new_query.upper().startswith("COMPLETE"):
                break

            # Perform a new search with the refined query
            current_query = new_query
            new_docs = self.vector_store.similarity_search(current_query)

            if new_docs:
                new_context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(new_docs)])
                # Update context with new information
                context = f"{context}\n\nAdditional Context:\n{new_context}"

        # Final synthesis
        final_prompt = f"""Original query: {user_query}

        Iterations of responses:
        {' '.join([f"Iteration {i+1}: {resp}" for i, resp in enumerate(iteration_responses)])}

        Please provide a final, comprehensive response to the original query that synthesizes all the information gathered across iterations.
        """

        final_response = self.llm.generate(final_prompt, system_message)

        return final_response

    def _should_continue(self, query: str, last_response: str) -> bool:
        """Determine if the agent should continue iterating."""
        prompt = f"""Query: {query}

        Current response:
        {last_response}

        Does this response fully address the query? If yes, respond with "COMPLETE". If not, respond with "CONTINUE" and briefly explain why.
        """

        decision = self.llm.generate(prompt)
        return "CONTINUE" in decision.upper()


# Command-line interface for the AgenticRAG system
def main():
    config = RAGConfig()
    rag_system = AgenticRAG(config)

    while True:
        print("\nAgenticRAG with DeepSeek")
        print("1. Ingest documents")
        print("2. Ask a question")
        print("3. Exit")

        choice = input("Enter your choice (1-3): ")

        if choice == "1":
            directory = input("Enter the directory path containing your documents: ")
            try:
                rag_system.ingest_documents(directory)
            except Exception as e:
                print(f"Error ingesting documents: {e}")

        elif choice == "2":
            query = input("Enter your question: ")
            try:
                start_time = time.time()
                response = rag_system.query(query)
                end_time = time.time()

                print(f"\nResponse (took {end_time - start_time:.2f} seconds):")
                print(response)
            except Exception as e:
                print(f"Error processing query: {e}")

        elif choice == "3":
            print("Thank you for using AgenticRAG with DeepSeek. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): 1
Enter the directory path containing your documents: /content/Document
Loading documents from /content/Document...
Loaded 1 documents.
Processing documents...
Created 4 chunks.
Creating vector store...


Vector store created successfully.

AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): 2
Enter your question: Morocco is known to be inhabited ?
Query: Morocco is known to be inhabited ?
Error processing query: 'FAISS' object has no attribute 'similarity_search_by_vector_with_relevance_scores'

AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): what is the population of Morocco ?
Invalid choice. Please try again.

AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): 1
Enter the directory path containing your documents: what is the population of Morocco ?
Loading documents from what is the population of Morocco ?...
Error loading *.txt documents: Directory not found: 'what is the population of Morocco ?'
Error loading *.pdf documents: Directory not found: 'what is the population of Morocco ?'
Error loading *.md documents: Directory not found: 'what is the 

KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import re
import time
import json
import requests
from typing import List, Dict, Any, Optional, Tuple
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    TextLoader,
    PDFMinerLoader,
    UnstructuredMarkdownLoader,
    CSVLoader
)
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.schema import Document


# Configuration class for the RAG system
class RAGConfig:
    def __init__(self):
        # DeepSeek API configuration (replace with your API key)
        self.deepseek_api_key = "sk-c6e6fbf20e3743dc98e51280d199b33d"
        self.deepseek_api_url = "https://api.deepseek.com/v1/chat/completions"

        # DeepSeek embedding model
        self.embedding_model_name = "deepseek-ai/deepseek-coder-7b-base"

        # Vector database configuration
        self.vector_db_path = "vector_db"

        # Document processing configuration
        self.chunk_size = 1000
        self.chunk_overlap = 200

        # Search configuration
        self.top_k = 5
        self.similarity_threshold = 0.7

        # Agent configuration
        self.max_iterations = 5
        self.thinking_steps = True


# Embedding class for document and query encoding
class DeepSeekEmbedding:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.embedding_model_name)
        self.model = AutoModel.from_pretrained(config.embedding_model_name)

    def get_embeddings(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for a list of texts."""
        embeddings = []

        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            with torch.no_grad():
                outputs = self.model(**inputs)

            # Use the mean of the last hidden state as the embedding
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy().tolist()
            embeddings.append(embedding)

        return embeddings


# Document processor for loading and chunking documents
class DocumentProcessor:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=config.chunk_size,
            chunk_overlap=config.chunk_overlap
        )

    def load_documents(self, directory_path: str) -> List[Document]:
        """Load documents from a directory."""
        # Configure loaders for different file types
        loaders = {
            "*.txt": TextLoader,
            "*.pdf": PDFMinerLoader,
            "*.md": UnstructuredMarkdownLoader,
            "*.csv": CSVLoader
        }

        documents = []

        for glob_pattern, loader_cls in loaders.items():
            try:
                loader = DirectoryLoader(
                    directory_path,
                    glob=glob_pattern,
                    loader_cls=loader_cls
                )
                docs = loader.load()
                documents.extend(docs)
            except Exception as e:
                print(f"Error loading {glob_pattern} documents: {e}")

        return documents

    def process_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks for embedding."""
        chunks = []

        for doc in documents:
            try:
                doc_chunks = self.text_splitter.split_documents([doc])
                chunks.extend(doc_chunks)
            except Exception as e:
                print(f"Error processing document {doc.metadata.get('source', 'unknown')}: {e}")

        return chunks


# Vector store for document storage and retrieval
class VectorStore:
    def __init__(self, config: RAGConfig, embedding_model: DeepSeekEmbedding):
        self.config = config
        self.embedding_model = embedding_model
        self.vector_store = None

    def create_vector_store(self, documents: List[Document]) -> None:
        """Create a vector store from documents."""
        texts = [doc.page_content for doc in documents]
        metadatas = [doc.metadata for doc in documents]

        embeddings = self.embedding_model.get_embeddings(texts)

        self.vector_store = FAISS.from_embeddings(
            text_embeddings=zip(texts, embeddings),
            embedding=self.embedding_model,
            metadatas=metadatas
        )

        # Save the vector store
        self.vector_store.save_local(self.config.vector_db_path)

    def load_vector_store(self) -> bool:
        """Load the vector store if it exists."""
        if os.path.exists(self.config.vector_db_path):
            self.vector_store = FAISS.load_local(
                self.config.vector_db_path,
                self.embedding_model
            )
            return True
        return False

# Replace the similarity_search method in the VectorStore class with this:
    def similarity_search(self, query: str) -> List[Document]:
        """Search for similar documents to the query."""
        if not self.vector_store:
            raise ValueError("Vector store not initialized. Please create or load a vector store first.")

        # Use the standard similarity_search method instead
        results = self.vector_store.similarity_search_with_score(
            query,
            k=self.config.top_k
        )

        # Filter results by similarity threshold
        # Note: FAISS returns distance, not similarity score, so we need to convert
        # FAISS distance is lower for more similar items
        filtered_results = [
            doc for doc, score in results
            if 1.0 / (1.0 + score) >= self.config.similarity_threshold  # Convert distance to similarity
        ]

        return filtered_results


# DeepSeek LLM client for RAG
class DeepSeekLLM:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {config.deepseek_api_key}"
        }

    def generate(self, prompt: str, system_message: str = None) -> str:
        """Generate text using DeepSeek API."""
        messages = []

        if system_message:
            messages.append({"role": "system", "content": system_message})

        messages.append({"role": "user", "content": prompt})

        data = {
            "model": "deepseek-chat",
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 2000
        }

        try:
            response = requests.post(
                self.config.deepseek_api_url,
                headers=self.headers,
                json=data
            )
            response.raise_for_status()

            result = response.json()
            return result["choices"][0]["message"]["content"]
        except Exception as e:
            print(f"Error calling DeepSeek API: {e}")
            return f"Error generating response: {str(e)}"


# Agentic RAG system that combines all components
class AgenticRAG:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.embedding_model = DeepSeekEmbedding(config)
        self.document_processor = DocumentProcessor(config)
        self.vector_store = VectorStore(config, self.embedding_model)
        self.llm = DeepSeekLLM(config)

    def ingest_documents(self, directory_path: str) -> None:
        """Ingest documents from a directory."""
        print(f"Loading documents from {directory_path}...")
        documents = self.document_processor.load_documents(directory_path)
        print(f"Loaded {len(documents)} documents.")

        print("Processing documents...")
        chunks = self.document_processor.process_documents(documents)
        print(f"Created {len(chunks)} chunks.")

        print("Creating vector store...")
        self.vector_store.create_vector_store(chunks)
        print("Vector store created successfully.")

    def query(self, user_query: str) -> str:
        """Process a user query and generate a response."""
        # Try to load the vector store if not already loaded
        if not self.vector_store.vector_store:
            if not self.vector_store.load_vector_store():
                return "No documents have been ingested. Please ingest documents first."

        # Initial response
        print(f"Query: {user_query}")

        # Agentic thinking process
        response = self._agentic_process(user_query)

        return response

    def _agentic_process(self, user_query: str) -> str:
        """Execute the agentic process for responding to queries."""
        system_message = """You are an intelligent agent with access to a knowledge base.
        Your task is to provide accurate, relevant information based on the query and the retrieved context.
        Think step by step and analyze the retrieved information carefully before formulating your final response."""

        # Initial retrieval
        retrieved_docs = self.vector_store.similarity_search(user_query)

        if not retrieved_docs:
            # Handle the case when no relevant documents are found
            prompt = f"""Query: {user_query}

            No relevant documents were found in the knowledge base. Please provide a general response based on your knowledge.
            """
            return self.llm.generate(prompt, system_message)

        # For agentic reasoning, we'll use a multi-step process
        context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

        iteration_responses = []
        current_query = user_query

        for iteration in range(self.config.max_iterations):
            # Check if we should continue
            if iteration > 0 and not self._should_continue(current_query, iteration_responses[-1]):
                break

            # Generate thinking steps if enabled
            thinking = ""
            if self.config.thinking_steps:
                thinking_prompt = f"""Query: {current_query}

                Context:
                {context}

                Think step by step about this query. What are the key points to address? What information from the context is most relevant? What additional information might be needed?
                """
                thinking = self.llm.generate(thinking_prompt, system_message)

            # Generate the response
            response_prompt = f"""Query: {current_query}

            Context:
            {context}

            {thinking if thinking else ""}

            Based on the context provided, please answer the query. If the context doesn't contain enough information, acknowledge this and provide the best answer you can.
            """

            response = self.llm.generate(response_prompt, system_message)
            iteration_responses.append(response)

            # Generate follow-up questions or refinements
            refinement_prompt = f"""Query: {current_query}

            Your current response:
            {response}

            Are there aspects of the query that haven't been fully addressed? What follow-up questions would help provide a more complete answer? How could the search be refined?
            """

            refinement = self.llm.generate(refinement_prompt, system_message)

            # Extract a new query for the next iteration
            new_query_prompt = f"""Original query: {user_query}

            Current response:
            {response}

            Refinement thoughts:
            {refinement}

            Based on the above, formulate a new search query that would help address any gaps in the current response. Return ONLY the new query without any explanation.
            If you believe the query has been fully addressed, return "COMPLETE".
            """

            new_query = self.llm.generate(new_query_prompt, system_message).strip()

            if new_query == "COMPLETE" or new_query.upper().startswith("COMPLETE"):
                break

            # Perform a new search with the refined query
            current_query = new_query
            new_docs = self.vector_store.similarity_search(current_query)

            if new_docs:
                new_context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(new_docs)])
                # Update context with new information
                context = f"{context}\n\nAdditional Context:\n{new_context}"

        # Final synthesis
        final_prompt = f"""Original query: {user_query}

        Iterations of responses:
        {' '.join([f"Iteration {i+1}: {resp}" for i, resp in enumerate(iteration_responses)])}

        Please provide a final, comprehensive response to the original query that synthesizes all the information gathered across iterations.
        """

        final_response = self.llm.generate(final_prompt, system_message)

        return final_response

    def _should_continue(self, query: str, last_response: str) -> bool:
        """Determine if the agent should continue iterating."""
        prompt = f"""Query: {query}

        Current response:
        {last_response}

        Does this response fully address the query? If yes, respond with "COMPLETE". If not, respond with "CONTINUE" and briefly explain why.
        """

        decision = self.llm.generate(prompt)
        return "CONTINUE" in decision.upper()


# Command-line interface for the AgenticRAG system
def main():
    config = RAGConfig()
    rag_system = AgenticRAG(config)

    while True:
        print("\nAgenticRAG with DeepSeek")
        print("1. Ingest documents")
        print("2. Ask a question")
        print("3. Exit")

        choice = input("Enter your choice (1-3): ")

        if choice == "1":
            directory = input("Enter the directory path containing your documents: ")
            try:
                rag_system.ingest_documents(directory)
            except Exception as e:
                print(f"Error ingesting documents: {e}")

        elif choice == "2":
            query = input("Enter your question: ")
            try:
                start_time = time.time()
                response = rag_system.query(query)
                end_time = time.time()

                print(f"\nResponse (took {end_time - start_time:.2f} seconds):")
                print(response)
            except Exception as e:
                print(f"Error processing query: {e}")

        elif choice == "3":
            print("Thank you for using AgenticRAG with DeepSeek. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): 1
Enter the directory path containing your documents: /content/Document
Loading documents from /content/Document...
Loaded 1 documents.
Processing documents...
Created 4 chunks.
Creating vector store...


Vector store created successfully.

AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): 2
Enter your question: hi
Query: hi
Error processing query: 'DeepSeekEmbedding' object is not callable

AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit


KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import re
import time
import json
import requests
from typing import List, Dict, Any, Optional, Tuple
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    TextLoader,
    PDFMinerLoader,
    UnstructuredMarkdownLoader,
    CSVLoader
)
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.schema import Document
from langchain.embeddings.base import Embeddings


# Configuration class for the RAG system
class RAGConfig:
    def __init__(self):
        # DeepSeek API configuration (replace with your API key)
        self.deepseek_api_key = "sk-c6e6fbf20e3743dc98e51280d199b33d"
        self.deepseek_api_url = "https://api.deepseek.com/v1/chat/completions"

        # DeepSeek embedding model
        self.embedding_model_name = "deepseek-ai/deepseek-coder-7b-base"

        # Vector database configuration
        self.vector_db_path = "vector_db"

        # Document processing configuration
        self.chunk_size = 1000
        self.chunk_overlap = 200

        # Search configuration
        self.top_k = 5
        self.similarity_threshold = 0.7

        # Agent configuration
        self.max_iterations = 5
        self.thinking_steps = True


# Embedding class for document and query encoding - implement Embeddings interface
class DeepSeekEmbedding(Embeddings):
    def __init__(self, config: RAGConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.embedding_model_name)
        self.model = AutoModel.from_pretrained(config.embedding_model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for a list of documents."""
        return self._get_embeddings(texts)

    def embed_query(self, text: str) -> List[float]:
        """Generate embedding for a query string."""
        embeddings = self._get_embeddings([text])
        return embeddings[0]

    def _get_embeddings(self, texts: List[str]) -> List[List[float]]:
        """Internal method to generate embeddings for a list of texts."""
        embeddings = []

        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            with torch.no_grad():
                outputs = self.model(**inputs)

            # Use the mean of the last hidden state as the embedding
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy().tolist()
            embeddings.append(embedding)

        return embeddings


# Document processor for loading and chunking documents
class DocumentProcessor:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=config.chunk_size,
            chunk_overlap=config.chunk_overlap
        )

    def load_documents(self, directory_path: str) -> List[Document]:
        """Load documents from a directory."""
        # Configure loaders for different file types
        loaders = {
            "*.txt": TextLoader,
            "*.pdf": PDFMinerLoader,
            "*.md": UnstructuredMarkdownLoader,
            "*.csv": CSVLoader
        }

        documents = []

        for glob_pattern, loader_cls in loaders.items():
            try:
                loader = DirectoryLoader(
                    directory_path,
                    glob=glob_pattern,
                    loader_cls=loader_cls
                )
                docs = loader.load()
                documents.extend(docs)
            except Exception as e:
                print(f"Error loading {glob_pattern} documents: {e}")

        return documents

    def process_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks for embedding."""
        chunks = []

        for doc in documents:
            try:
                doc_chunks = self.text_splitter.split_documents([doc])
                chunks.extend(doc_chunks)
            except Exception as e:
                print(f"Error processing document {doc.metadata.get('source', 'unknown')}: {e}")

        return chunks


# Vector store for document storage and retrieval
class VectorStore:
    def __init__(self, config: RAGConfig, embedding_model: DeepSeekEmbedding):
        self.config = config
        self.embedding_model = embedding_model
        self.vector_store = None

    def create_vector_store(self, documents: List[Document]) -> None:
        """Create a vector store from documents."""
        # Use the FAISS.from_documents method which properly handles Embeddings objects
        self.vector_store = FAISS.from_documents(
            documents,
            self.embedding_model
        )

        # Save the vector store
        self.vector_store.save_local(self.config.vector_db_path)

    def load_vector_store(self) -> bool:
        """Load the vector store if it exists."""
        if os.path.exists(self.config.vector_db_path):
            self.vector_store = FAISS.load_local(
                self.config.vector_db_path,
                self.embedding_model
            )
            return True
        return False

    def similarity_search(self, query: str) -> List[Document]:
        """Search for similar documents to the query."""
        if not self.vector_store:
            raise ValueError("Vector store not initialized. Please create or load a vector store first.")

        # Use the standard similarity_search method instead
        results = self.vector_store.similarity_search_with_score(
            query,
            k=self.config.top_k
        )

        # Filter results by similarity threshold
        # Note: FAISS returns distance, not similarity score, so we need to convert
        # FAISS distance is lower for more similar items
        filtered_results = [
            doc for doc, score in results
            if 1.0 / (1.0 + score) >= self.config.similarity_threshold  # Convert distance to similarity
        ]

        return filtered_results


# DeepSeek LLM client for RAG
class DeepSeekLLM:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {config.deepseek_api_key}"
        }

    def generate(self, prompt: str, system_message: str = None) -> str:
        """Generate text using DeepSeek API."""
        messages = []

        if system_message:
            messages.append({"role": "system", "content": system_message})

        messages.append({"role": "user", "content": prompt})

        data = {
            "model": "deepseek-chat",
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 2000
        }

        try:
            response = requests.post(
                self.config.deepseek_api_url,
                headers=self.headers,
                json=data
            )
            response.raise_for_status()

            result = response.json()
            return result["choices"][0]["message"]["content"]
        except Exception as e:
            print(f"Error calling DeepSeek API: {e}")
            return f"Error generating response: {str(e)}"


# Agentic RAG system that combines all components
class AgenticRAG:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.embedding_model = DeepSeekEmbedding(config)
        self.document_processor = DocumentProcessor(config)
        self.vector_store = VectorStore(config, self.embedding_model)
        self.llm = DeepSeekLLM(config)

    def ingest_documents(self, directory_path: str) -> None:
        """Ingest documents from a directory."""
        print(f"Loading documents from {directory_path}...")
        documents = self.document_processor.load_documents(directory_path)
        print(f"Loaded {len(documents)} documents.")

        print("Processing documents...")
        chunks = self.document_processor.process_documents(documents)
        print(f"Created {len(chunks)} chunks.")

        print("Creating vector store...")
        self.vector_store.create_vector_store(chunks)
        print("Vector store created successfully.")

    def query(self, user_query: str) -> str:
        """Process a user query and generate a response."""
        # Try to load the vector store if not already loaded
        if not self.vector_store.vector_store:
            if not self.vector_store.load_vector_store():
                return "No documents have been ingested. Please ingest documents first."

        # Initial response
        print(f"Query: {user_query}")

        # Agentic thinking process
        response = self._agentic_process(user_query)

        return response

    def _agentic_process(self, user_query: str) -> str:
        """Execute the agentic process for responding to queries."""
        system_message = """You are an intelligent agent with access to a knowledge base.
        Your task is to provide accurate, relevant information based on the query and the retrieved context.
        Think step by step and analyze the retrieved information carefully before formulating your final response."""

        # Initial retrieval
        retrieved_docs = self.vector_store.similarity_search(user_query)

        if not retrieved_docs:
            # Handle the case when no relevant documents are found
            prompt = f"""Query: {user_query}

            No relevant documents were found in the knowledge base. Please provide a general response based on your knowledge.
            """
            return self.llm.generate(prompt, system_message)

        # For agentic reasoning, we'll use a multi-step process
        context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

        iteration_responses = []
        current_query = user_query

        for iteration in range(self.config.max_iterations):
            # Check if we should continue
            if iteration > 0 and not self._should_continue(current_query, iteration_responses[-1]):
                break

            # Generate thinking steps if enabled
            thinking = ""
            if self.config.thinking_steps:
                thinking_prompt = f"""Query: {current_query}

                Context:
                {context}

                Think step by step about this query. What are the key points to address? What information from the context is most relevant? What additional information might be needed?
                """
                thinking = self.llm.generate(thinking_prompt, system_message)

            # Generate the response
            response_prompt = f"""Query: {current_query}

            Context:
            {context}

            {thinking if thinking else ""}

            Based on the context provided, please answer the query. If the context doesn't contain enough information, acknowledge this and provide the best answer you can.
            """

            response = self.llm.generate(response_prompt, system_message)
            iteration_responses.append(response)

            # Generate follow-up questions or refinements
            refinement_prompt = f"""Query: {current_query}

            Your current response:
            {response}

            Are there aspects of the query that haven't been fully addressed? What follow-up questions would help provide a more complete answer? How could the search be refined?
            """

            refinement = self.llm.generate(refinement_prompt, system_message)

            # Extract a new query for the next iteration
            new_query_prompt = f"""Original query: {user_query}

            Current response:
            {response}

            Refinement thoughts:
            {refinement}

            Based on the above, formulate a new search query that would help address any gaps in the current response. Return ONLY the new query without any explanation.
            If you believe the query has been fully addressed, return "COMPLETE".
            """

            new_query = self.llm.generate(new_query_prompt, system_message).strip()

            if new_query == "COMPLETE" or new_query.upper().startswith("COMPLETE"):
                break

            # Perform a new search with the refined query
            current_query = new_query
            new_docs = self.vector_store.similarity_search(current_query)

            if new_docs:
                new_context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(new_docs)])
                # Update context with new information
                context = f"{context}\n\nAdditional Context:\n{new_context}"

        # Final synthesis
        final_prompt = f"""Original query: {user_query}

        Iterations of responses:
        {' '.join([f"Iteration {i+1}: {resp}" for i, resp in enumerate(iteration_responses)])}

        Please provide a final, comprehensive response to the original query that synthesizes all the information gathered across iterations.
        """

        final_response = self.llm.generate(final_prompt, system_message)

        return final_response

    def _should_continue(self, query: str, last_response: str) -> bool:
        """Determine if the agent should continue iterating."""
        prompt = f"""Query: {query}

        Current response:
        {last_response}

        Does this response fully address the query? If yes, respond with "COMPLETE". If not, respond with "CONTINUE" and briefly explain why.
        """

        decision = self.llm.generate(prompt)
        return "CONTINUE" in decision.upper()


# Command-line interface for the AgenticRAG system
def main():
    config = RAGConfig()
    rag_system = AgenticRAG(config)

    while True:
        print("\nAgenticRAG with DeepSeek")
        print("1. Ingest documents")
        print("2. Ask a question")
        print("3. Exit")

        choice = input("Enter your choice (1-3): ")

        if choice == "1":
            directory = input("Enter the directory path containing your documents: ")
            try:
                rag_system.ingest_documents(directory)
            except Exception as e:
                print(f"Error ingesting documents: {e}")

        elif choice == "2":
            query = input("Enter your question: ")
            try:
                start_time = time.time()
                response = rag_system.query(query)
                end_time = time.time()

                print(f"\nResponse (took {end_time - start_time:.2f} seconds):")
                print(response)
            except Exception as e:
                print(f"Error processing query: {e}")

        elif choice == "3":
            print("Thank you for using AgenticRAG with DeepSeek. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): 1
Enter the directory path containing your documents: /content/Document
Loading documents from /content/Document...
Loaded 1 documents.
Processing documents...
Created 4 chunks.
Creating vector store...
Vector store created successfully.

AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): 2
Enter your question: what is the capital of moroco ?
Query: what is the capital of moroco ?
Error calling DeepSeek API: 402 Client Error: Payment Required for url: https://api.deepseek.com/v1/chat/completions

Response (took 12.90 seconds):
Error generating response: 402 Client Error: Payment Required for url: https://api.deepseek.com/v1/chat/completions

AgenticRAG with DeepSeek
1. Ingest documents
2. Ask a question
3. Exit


KeyboardInterrupt: Interrupted by user

In [12]:
import os
import re
import time
import json
from typing import List, Dict, Any, Optional, Tuple
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    TextLoader,
    PDFMinerLoader,
    UnstructuredMarkdownLoader,
    CSVLoader
)
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.schema import Document
from langchain.embeddings.base import Embeddings


# Configuration class for the RAG system
class RAGConfig:
    def __init__(self):
        # Local model configuration - using publicly available models
        self.embedding_model_name = "sentence-transformers/all-mpnet-base-v2"  # Public embedding model
        self.llm_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Public LLM model

        # Model parameters
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.torch_dtype = torch.float16 if self.device == "cuda" else torch.float32

        # Vector database configuration
        self.vector_db_path = "vector_db"

        # Document processing configuration
        self.chunk_size = 1000
        self.chunk_overlap = 200

        # Search configuration
        self.top_k = 5
        self.similarity_threshold = 0.7

        # Agent configuration
        self.max_iterations = 3  # Reduced for faster execution
        self.thinking_steps = True


# Embedding class for document and query encoding - implement Embeddings interface
class SentenceTransformerEmbedding(Embeddings):
    def __init__(self, config: RAGConfig):
        self.config = config
        print(f"Loading embedding model {config.embedding_model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(config.embedding_model_name)
        self.model = AutoModel.from_pretrained(
            config.embedding_model_name,
            torch_dtype=config.torch_dtype
        ).to(config.device)
        print("Embedding model loaded successfully")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for a list of documents."""
        return self._get_embeddings(texts)

    def embed_query(self, text: str) -> List[float]:
        """Generate embedding for a query string."""
        embeddings = self._get_embeddings([text])
        return embeddings[0]

    def _get_embeddings(self, texts: List[str]) -> List[List[float]]:
        """Internal method to generate embeddings for a list of texts."""
        embeddings = []

        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.config.device)
            with torch.no_grad():
                outputs = self.model(**inputs)

            # Use the mean of the last hidden state as the embedding
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy().tolist()
            embeddings.append(embedding)

        return embeddings


# Document processor for loading and chunking documents
class DocumentProcessor:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=config.chunk_size,
            chunk_overlap=config.chunk_overlap
        )

    def load_documents(self, directory_path: str) -> List[Document]:
        """Load documents from a directory."""
        # Configure loaders for different file types
        loaders = {
            "*.txt": TextLoader,
            "*.pdf": PDFMinerLoader,
            "*.md": UnstructuredMarkdownLoader,
            "*.csv": CSVLoader
        }

        documents = []

        for glob_pattern, loader_cls in loaders.items():
            try:
                loader = DirectoryLoader(
                    directory_path,
                    glob=glob_pattern,
                    loader_cls=loader_cls
                )
                docs = loader.load()
                documents.extend(docs)
            except Exception as e:
                print(f"Error loading {glob_pattern} documents: {e}")

        return documents

    def process_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks for embedding."""
        chunks = []

        for doc in documents:
            try:
                doc_chunks = self.text_splitter.split_documents([doc])
                chunks.extend(doc_chunks)
            except Exception as e:
                print(f"Error processing document {doc.metadata.get('source', 'unknown')}: {e}")

        return chunks


# Vector store for document storage and retrieval
class VectorStore:
    def __init__(self, config: RAGConfig, embedding_model: SentenceTransformerEmbedding):
        self.config = config
        self.embedding_model = embedding_model
        self.vector_store = None

    def create_vector_store(self, documents: List[Document]) -> None:
        """Create a vector store from documents."""
        # Use the FAISS.from_documents method which properly handles Embeddings objects
        self.vector_store = FAISS.from_documents(
            documents,
            self.embedding_model
        )

        # Save the vector store
        self.vector_store.save_local(self.config.vector_db_path)

    def load_vector_store(self) -> bool:
        """Load the vector store if it exists."""
        if os.path.exists(self.config.vector_db_path):
            self.vector_store = FAISS.load_local(
                self.config.vector_db_path,
                self.embedding_model
            )
            return True
        return False

    def similarity_search(self, query: str) -> List[Document]:
        """Search for similar documents to the query."""
        if not self.vector_store:
            raise ValueError("Vector store not initialized. Please create or load a vector store first.")

        # Use the standard similarity_search method instead
        results = self.vector_store.similarity_search_with_score(
            query,
            k=self.config.top_k
        )

        # Filter results by similarity threshold
        # Note: FAISS returns distance, not similarity score, so we need to convert
        # FAISS distance is lower for more similar items
        filtered_results = [
            doc for doc, score in results
            if 1.0 / (1.0 + score) >= self.config.similarity_threshold  # Convert distance to similarity
        ]

        return filtered_results


# Local LLM for RAG
class LocalLLM:
    def __init__(self, config: RAGConfig):
        self.config = config
        print(f"Loading LLM model {config.llm_model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(config.llm_model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            config.llm_model_name,
            torch_dtype=config.torch_dtype
        ).to(config.device)
        print(f"LLM model loaded successfully on {config.device}")

    def generate(self, prompt: str, system_message: str = None) -> str:
        """Generate text using local model."""
        # Format the messages
        if system_message:
            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt}
            ]
        else:
            messages = [{"role": "user", "content": prompt}]

        try:
            # Format messages for chat format
            formatted_prompt = self.tokenizer.apply_chat_template(messages, tokenize=False)
        except Exception as e:
            # Fallback if the model doesn't support chat templates
            print(f"Chat template error: {e}, using simple prompt formatting")
            formatted_prompt = system_message + "\n\n" + prompt if system_message else prompt

        # Tokenize and generate
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.config.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                max_new_tokens=1000,  # Reduced for faster generation
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        # Decode the output, removing the input tokens
        input_length = inputs.input_ids.shape[1]
        response_tokens = outputs[0][input_length:]
        response = self.tokenizer.decode(response_tokens, skip_special_tokens=True)

        return response


# Agentic RAG system that combines all components
class AgenticRAG:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.embedding_model = SentenceTransformerEmbedding(config)
        self.document_processor = DocumentProcessor(config)
        self.vector_store = VectorStore(config, self.embedding_model)
        self.llm = LocalLLM(config)

    def ingest_documents(self, directory_path: str) -> None:
        """Ingest documents from a directory."""
        print(f"Loading documents from {directory_path}...")
        documents = self.document_processor.load_documents(directory_path)
        print(f"Loaded {len(documents)} documents.")

        print("Processing documents...")
        chunks = self.document_processor.process_documents(documents)
        print(f"Created {len(chunks)} chunks.")

        print("Creating vector store...")
        self.vector_store.create_vector_store(chunks)
        print("Vector store created successfully.")

    def query(self, user_query: str) -> str:
        """Process a user query and generate a response."""
        # Try to load the vector store if not already loaded
        if not self.vector_store.vector_store:
            if not self.vector_store.load_vector_store():
                return "No documents have been ingested. Please ingest documents first."

        # Initial response
        print(f"Query: {user_query}")

        # Agentic thinking process
        response = self._agentic_process(user_query)

        return response

    def _agentic_process(self, user_query: str) -> str:
        """Execute the agentic process for responding to queries."""
        system_message = """You are an intelligent agent with access to a knowledge base.
        Your task is to provide accurate, relevant information based on the query and the retrieved context.
        Think step by step and analyze the retrieved information carefully before formulating your final response."""

        # Initial retrieval
        retrieved_docs = self.vector_store.similarity_search(user_query)

        if not retrieved_docs:
            # Handle the case when no relevant documents are found
            prompt = f"""Query: {user_query}

            No relevant documents were found in the knowledge base. Please provide a general response based on your knowledge.
            """
            return self.llm.generate(prompt, system_message)

        # For agentic reasoning, we'll use a multi-step process
        context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

        iteration_responses = []
        current_query = user_query

        for iteration in range(self.config.max_iterations):
            print(f"Iteration {iteration+1}/{self.config.max_iterations}")

            # Check if we should continue
            if iteration > 0 and not self._should_continue(current_query, iteration_responses[-1]):
                break

            # Generate thinking steps if enabled
            thinking = ""
            if self.config.thinking_steps:
                thinking_prompt = f"""Query: {current_query}

                Context:
                {context}

                Think step by step about this query. What are the key points to address? What information from the context is most relevant? What additional information might be needed?
                """
                thinking = self.llm.generate(thinking_prompt, system_message)

            # Generate the response
            response_prompt = f"""Query: {current_query}

            Context:
            {context}

            {thinking if thinking else ""}

            Based on the context provided, please answer the query. If the context doesn't contain enough information, acknowledge this and provide the best answer you can.
            """

            response = self.llm.generate(response_prompt, system_message)
            iteration_responses.append(response)

            # Generate follow-up questions or refinements
            refinement_prompt = f"""Query: {current_query}

            Your current response:
            {response}

            Are there aspects of the query that haven't been fully addressed? What follow-up questions would help provide a more complete answer? How could the search be refined?
            """

            refinement = self.llm.generate(refinement_prompt, system_message)

            # Extract a new query for the next iteration
            new_query_prompt = f"""Original query: {user_query}

            Current response:
            {response}

            Refinement thoughts:
            {refinement}

            Based on the above, formulate a new search query that would help address any gaps in the current response. Return ONLY the new query without any explanation.
            If you believe the query has been fully addressed, return "COMPLETE".
            """

            new_query = self.llm.generate(new_query_prompt, system_message).strip()

            if new_query == "COMPLETE" or new_query.upper().startswith("COMPLETE"):
                break

            # Perform a new search with the refined query
            current_query = new_query
            new_docs = self.vector_store.similarity_search(current_query)

            if new_docs:
                new_context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(new_docs)])
                # Update context with new information
                context = f"{context}\n\nAdditional Context:\n{new_context}"

        # Final synthesis
        final_prompt = f"""Original query: {user_query}

        Iterations of responses:
        {' '.join([f"Iteration {i+1}: {resp}" for i, resp in enumerate(iteration_responses)])}

        Please provide a final, comprehensive response to the original query that synthesizes all the information gathered across iterations.
        """

        final_response = self.llm.generate(final_prompt, system_message)

        return final_response

    def _should_continue(self, query: str, last_response: str) -> bool:
        """Determine if the agent should continue iterating."""
        prompt = f"""Query: {query}

        Current response:
        {last_response}

        Does this response fully address the query? If yes, respond with "COMPLETE". If not, respond with "CONTINUE" and briefly explain why.
        """

        decision = self.llm.generate(prompt)
        return "CONTINUE" in decision.upper()


# Command-line interface for the AgenticRAG system
def main():
    config = RAGConfig()
    print(f"Initializing AgenticRAG with models on {config.device}")
    rag_system = AgenticRAG(config)

    while True:
        print("\nAgenticRAG with Local LLM")
        print("1. Ingest documents")
        print("2. Ask a question")
        print("3. Exit")

        choice = input("Enter your choice (1-3): ")

        if choice == "1":
            directory = input("Enter the directory path containing your documents: ")
            try:
                rag_system.ingest_documents(directory)
            except Exception as e:
                print(f"Error ingesting documents: {e}")

        elif choice == "2":
            query = input("Enter your question: ")
            try:
                start_time = time.time()
                response = rag_system.query(query)
                end_time = time.time()

                print(f"\nResponse (took {end_time - start_time:.2f} seconds):")
                print(response)
            except Exception as e:
                print(f"Error processing query: {e}")

        elif choice == "3":
            print("Thank you for using AgenticRAG with Local LLM. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()

Initializing AgenticRAG with models on cpu
Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu

AgenticRAG with Local LLM
1. Ingest documents
2. Ask a question
3. Exit
Enter your choice (1-3): 1
Enter the directory path containing your documents: /content/c._r._pennell_morocco_from_empire_to_independenbook4me.org_.pdf
Loading documents from /content/c._r._pennell_morocco_from_empire_to_independenbook4me.org_.pdf...
Error loading *.txt documents: Expected directory, got file: '/content/c._r._pennell_morocco_from_empire_to_independenbook4me.org_.pdf'
Error loading *.pdf documents: Expected directory, got file: '/content/c._r._pennell_morocco_from_empire_to_independenbook4me.org_.pdf'
Error loading *.md documents: Expected directory, got file: '/content/c._r._pennell_morocco_from_empire_to_independenbook4me.org_.pdf'
Error loading *.csv documents:

KeyboardInterrupt: Interrupted by user

In [15]:
import os
import re
import time
import json
import tempfile
from typing import List, Dict, Any, Optional, Tuple, BinaryIO
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    TextLoader,
    PDFMinerLoader,
    UnstructuredMarkdownLoader,
    CSVLoader
)
from langchain.schema import Document
from langchain.embeddings.base import Embeddings


# Configuration class for the RAG system
class RAGConfig:
    def __init__(self):
        # Local model configuration - using publicly available models
        self.embedding_model_name = "sentence-transformers/all-mpnet-base-v2"  # Public embedding model
        self.llm_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Public LLM model

        # Model parameters
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.torch_dtype = torch.float16 if self.device == "cuda" else torch.float32

        # Vector database configuration
        self.vector_db_path = "vector_db"

        # Document processing configuration
        self.chunk_size = 1000
        self.chunk_overlap = 200

        # Search configuration
        self.top_k = 5
        self.similarity_threshold = 0.7

        # Agent configuration
        self.max_iterations = 3  # Reduced for faster execution
        self.thinking_steps = True


# Embedding class for document and query encoding - implement Embeddings interface
class SentenceTransformerEmbedding(Embeddings):
    def __init__(self, config: RAGConfig):
        self.config = config
        print(f"Loading embedding model {config.embedding_model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(config.embedding_model_name)
        self.model = AutoModel.from_pretrained(
            config.embedding_model_name,
            torch_dtype=config.torch_dtype
        ).to(config.device)
        print("Embedding model loaded successfully")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for a list of documents."""
        return self._get_embeddings(texts)

    def embed_query(self, text: str) -> List[float]:
        """Generate embedding for a query string."""
        embeddings = self._get_embeddings([text])
        return embeddings[0]

    def _get_embeddings(self, texts: List[str]) -> List[List[float]]:
        """Internal method to generate embeddings for a list of texts."""
        embeddings = []

        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.config.device)
            with torch.no_grad():
                outputs = self.model(**inputs)

            # Use the mean of the last hidden state as the embedding
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy().tolist()
            embeddings.append(embedding)

        return embeddings


# Document processor for loading and processing uploaded files
class DocumentProcessor:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=config.chunk_size,
            chunk_overlap=config.chunk_overlap
        )
        self.temp_dir = None

    def process_uploaded_file(self, file_obj: BinaryIO, filename: str) -> List[Document]:
        """Process a single uploaded file."""
        # Create a temporary directory if not already created
        if self.temp_dir is None:
            self.temp_dir = tempfile.mkdtemp()

        # Get file extension
        _, file_extension = os.path.splitext(filename)
        file_extension = file_extension.lower()

        # Save the file to the temporary directory
        temp_file_path = os.path.join(self.temp_dir, filename)
        with open(temp_file_path, 'wb') as f:
            f.write(file_obj.read())

        # Select appropriate loader based on file extension
        loader = None
        if file_extension == '.txt':
            loader = TextLoader(temp_file_path)
        elif file_extension == '.pdf':
            loader = PDFMinerLoader(temp_file_path)
        elif file_extension == '.md':
            loader = UnstructuredMarkdownLoader(temp_file_path)
        elif file_extension == '.csv':
            loader = CSVLoader(temp_file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_extension}")

        # Load and process the document
        documents = loader.load()
        chunks = self.process_documents(documents)

        return chunks

    def process_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks for embedding."""
        chunks = []

        for doc in documents:
            try:
                doc_chunks = self.text_splitter.split_documents([doc])
                chunks.extend(doc_chunks)
            except Exception as e:
                print(f"Error processing document {doc.metadata.get('source', 'unknown')}: {e}")

        return chunks

    def cleanup(self):
        """Remove temporary files when done."""
        if self.temp_dir and os.path.exists(self.temp_dir):
            import shutil
            shutil.rmtree(self.temp_dir)
            self.temp_dir = None


# Vector store for document storage and retrieval
class VectorStore:
    def __init__(self, config: RAGConfig, embedding_model: SentenceTransformerEmbedding):
        self.config = config
        self.embedding_model = embedding_model
        self.vector_store = None

    def create_vector_store(self, documents: List[Document]) -> None:
        """Create a vector store from documents."""
        # Use the FAISS.from_documents method which properly handles Embeddings objects
        self.vector_store = FAISS.from_documents(
            documents,
            self.embedding_model
        )

        # Save the vector store
        self.vector_store.save_local(self.config.vector_db_path)

    def add_documents(self, documents: List[Document]) -> None:
        """Add documents to an existing vector store."""
        if self.vector_store is None:
            # If no vector store exists, create a new one
            self.create_vector_store(documents)
        else:
            # Add documents to existing vector store
            self.vector_store.add_documents(documents)
            # Save the updated vector store
            self.vector_store.save_local(self.config.vector_db_path)

    def load_vector_store(self) -> bool:
        """Load the vector store if it exists."""
        if os.path.exists(self.config.vector_db_path):
            self.vector_store = FAISS.load_local(
                self.config.vector_db_path,
                self.embedding_model
            )
            return True
        return False

    def similarity_search(self, query: str) -> List[Document]:
        """Search for similar documents to the query."""
        if not self.vector_store:
            raise ValueError("Vector store not initialized. Please create or load a vector store first.")

        # Use the standard similarity_search method instead
        results = self.vector_store.similarity_search_with_score(
            query,
            k=self.config.top_k
        )

        # Filter results by similarity threshold
        # Note: FAISS returns distance, not similarity score, so we need to convert
        # FAISS distance is lower for more similar items
        filtered_results = [
            doc for doc, score in results
            if 1.0 / (1.0 + score) >= self.config.similarity_threshold  # Convert distance to similarity
        ]

        return filtered_results


# Local LLM for RAG
class LocalLLM:
    def __init__(self, config: RAGConfig):
        self.config = config
        print(f"Loading LLM model {config.llm_model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(config.llm_model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            config.llm_model_name,
            torch_dtype=config.torch_dtype
        ).to(config.device)
        print(f"LLM model loaded successfully on {config.device}")

    def generate(self, prompt: str, system_message: str = None) -> str:
        """Generate text using local model."""
        # Format the messages
        if system_message:
            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt}
            ]
        else:
            messages = [{"role": "user", "content": prompt}]

        try:
            # Format messages for chat format
            formatted_prompt = self.tokenizer.apply_chat_template(messages, tokenize=False)
        except Exception as e:
            # Fallback if the model doesn't support chat templates
            print(f"Chat template error: {e}, using simple prompt formatting")
            formatted_prompt = system_message + "\n\n" + prompt if system_message else prompt

        # Tokenize and generate
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.config.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                max_new_tokens=1000,  # Reduced for faster generation
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        # Decode the output, removing the input tokens
        input_length = inputs.input_ids.shape[1]
        response_tokens = outputs[0][input_length:]
        response = self.tokenizer.decode(response_tokens, skip_special_tokens=True)

        return response


# Agentic RAG system that combines all components
class AgenticRAG:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.embedding_model = SentenceTransformerEmbedding(config)
        self.document_processor = DocumentProcessor(config)
        self.vector_store = VectorStore(config, self.embedding_model)
        self.llm = LocalLLM(config)

    def ingest_uploaded_file(self, file_obj: BinaryIO, filename: str) -> None:
        """Ingest a single uploaded file."""
        print(f"Processing uploaded file: {filename}...")

        try:
            # Process the uploaded file
            chunks = self.document_processor.process_uploaded_file(file_obj, filename)
            print(f"Created {len(chunks)} chunks from {filename}.")

            # Try to load the vector store first
            vector_store_exists = self.vector_store.load_vector_store()

            if vector_store_exists:
                # Add the new documents to the existing vector store
                print("Adding documents to existing vector store...")
                self.vector_store.add_documents(chunks)
            else:
                # Create a new vector store if none exists
                print("Creating new vector store...")
                self.vector_store.create_vector_store(chunks)

            print(f"Successfully ingested {filename}.")

        except Exception as e:
            print(f"Error ingesting file {filename}: {e}")
            raise

    def query(self, user_query: str) -> str:
        """Process a user query and generate a response."""
        # Try to load the vector store if not already loaded
        if not self.vector_store.vector_store:
            if not self.vector_store.load_vector_store():
                return "No documents have been ingested. Please ingest documents first."

        # Initial response
        print(f"Query: {user_query}")

        # Agentic thinking process
        response = self._agentic_process(user_query)

        return response

    def _agentic_process(self, user_query: str) -> str:
        """Execute the agentic process for responding to queries."""
        system_message = """You are an intelligent agent with access to a knowledge base.
        Your task is to provide accurate, relevant information based on the query and the retrieved context.
        Think step by step and analyze the retrieved information carefully before formulating your final response."""

        # Initial retrieval
        retrieved_docs = self.vector_store.similarity_search(user_query)

        if not retrieved_docs:
            # Handle the case when no relevant documents are found
            prompt = f"""Query: {user_query}

            No relevant documents were found in the knowledge base. Please provide a general response based on your knowledge.
            """
            return self.llm.generate(prompt, system_message)

        # For agentic reasoning, we'll use a multi-step process
        context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

        iteration_responses = []
        current_query = user_query

        for iteration in range(self.config.max_iterations):
            print(f"Iteration {iteration+1}/{self.config.max_iterations}")

            # Check if we should continue
            if iteration > 0 and not self._should_continue(current_query, iteration_responses[-1]):
                break

            # Generate thinking steps if enabled
            thinking = ""
            if self.config.thinking_steps:
                thinking_prompt = f"""Query: {current_query}

                Context:
                {context}

                Think step by step about this query. What are the key points to address? What information from the context is most relevant? What additional information might be needed?
                """
                thinking = self.llm.generate(thinking_prompt, system_message)

            # Generate the response
            response_prompt = f"""Query: {current_query}

            Context:
            {context}

            {thinking if thinking else ""}

            Based on the context provided, please answer the query. If the context doesn't contain enough information, acknowledge this and provide the best answer you can.
            """

            response = self.llm.generate(response_prompt, system_message)
            iteration_responses.append(response)

            # Generate follow-up questions or refinements
            refinement_prompt = f"""Query: {current_query}

            Your current response:
            {response}

            Are there aspects of the query that haven't been fully addressed? What follow-up questions would help provide a more complete answer? How could the search be refined?
            """

            refinement = self.llm.generate(refinement_prompt, system_message)

            # Extract a new query for the next iteration
            new_query_prompt = f"""Original query: {user_query}

            Current response:
            {response}

            Refinement thoughts:
            {refinement}

            Based on the above, formulate a new search query that would help address any gaps in the current response. Return ONLY the new query without any explanation.
            If you believe the query has been fully addressed, return "COMPLETE".
            """

            new_query = self.llm.generate(new_query_prompt, system_message).strip()

            if new_query == "COMPLETE" or new_query.upper().startswith("COMPLETE"):
                break

            # Perform a new search with the refined query
            current_query = new_query
            new_docs = self.vector_store.similarity_search(current_query)

            if new_docs:
                new_context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(new_docs)])
                # Update context with new information
                context = f"{context}\n\nAdditional Context:\n{new_context}"

        # Final synthesis
        final_prompt = f"""Original query: {user_query}

        Iterations of responses:
        {' '.join([f"Iteration {i+1}: {resp}" for i, resp in enumerate(iteration_responses)])}

        Please provide a final, comprehensive response to the original query that synthesizes all the information gathered across iterations.
        """

        final_response = self.llm.generate(final_prompt, system_message)

        return final_response

    def _should_continue(self, query: str, last_response: str) -> bool:
        """Determine if the agent should continue iterating."""
        prompt = f"""Query: {query}

        Current response:
        {last_response}

        Does this response fully address the query? If yes, respond with "COMPLETE". If not, respond with "CONTINUE" and briefly explain why.
        """

        decision = self.llm.generate(prompt)
        return "CONTINUE" in decision.upper()

    def cleanup(self):
        """Clean up temporary files."""
        self.document_processor.cleanup()


# Google Colab integration for file upload and RAG system
def run_in_colab():
    from google.colab import files
    import io

    config = RAGConfig()
    print(f"Initializing AgenticRAG with models on {config.device}")
    rag_system = AgenticRAG(config)

    while True:
        print("\nAgenticRAG with Local LLM")
        print("1. Upload and ingest a file")
        print("2. Ask a question")
        print("3. Exit")

        choice = input("Enter your choice (1-3): ")

        if choice == "1":
            try:
                print("Please select a file to upload...")
                uploaded = files.upload()

                for filename, content in uploaded.items():
                    file_obj = io.BytesIO(content)
                    rag_system.ingest_uploaded_file(file_obj, filename)
            except Exception as e:
                print(f"Error uploading and ingesting file: {e}")

        elif choice == "2":
            query = input("Enter your question: ")
            try:
                start_time = time.time()
                response = rag_system.query(query)
                end_time = time.time()

                print(f"\nResponse (took {end_time - start_time:.2f} seconds):")
                print(response)
            except Exception as e:
                print(f"Error processing query: {e}")

        elif choice == "3":
            print("Thank you for using AgenticRAG with Local LLM. Goodbye!")
            # Clean up any temporary files
            rag_system.cleanup()
            break

        else:
            print("Invalid choice. Please try again.")


# For standalone execution outside of Colab
def main():
    try:
        # Check if running in Google Colab
        import google.colab
        print("Running in Google Colab environment")
        run_in_colab()
    except ImportError:
        print("Not running in Google Colab, falling back to command-line interface")
        # Original command-line interface code here
        # (You can keep the original code if needed, but it's not the focus now)


if __name__ == "__main__":
    main()

Running in Google Colab environment
Initializing AgenticRAG with models on cpu
Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu

AgenticRAG with Local LLM
1. Upload and ingest a file
2. Ask a question
3. Exit
Please select a file to upload...


Saving 1706.03762v7 (1).pdf to 1706.03762v7 (1).pdf
Processing uploaded file: 1706.03762v7 (1).pdf...


Created 52 chunks from 1706.03762v7 (1).pdf.
Creating new vector store...
Successfully ingested 1706.03762v7 (1).pdf.

AgenticRAG with Local LLM
1. Upload and ingest a file
2. Ask a question
3. Exit
Query: what is attention


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Response (took 360.28 seconds):
<|assistant|>
Attention is a crucial cognitive function that helps us process and interpret information. When we encounter a new piece of information, our brains scan it for relevant details to help us understand its meaning. For example, if we receive an email with a detailed product description, our attention is captured and we focus on details such as the product name, pictures, and price.

In this scenario, your agent should respond by providing an overview of the product description, such as the name, brand, features, and price. A general response might include something like, "that's a great description of the product. Would you like to know more about its features and price?"

Your agent should think carefully about the context of the query and the retrieved information to provide a more detailed response. You could also ask follow-up questions to clarify the information provided by the context. For example, "Would you like to know more about the

KeyboardInterrupt: Interrupted by user

In [10]:
import os
import re
import time
import json
from typing import List, Dict, Any, Optional, Tuple
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    TextLoader,
    PDFMinerLoader,
    UnstructuredMarkdownLoader,
    CSVLoader
)
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.schema import Document
from langchain.embeddings.base import Embeddings

# [Previous code for RAGConfig, SentenceTransformerEmbedding, DocumentProcessor, VectorStore, LocalLLM remains the same]

# New Multi-Agent Communication System
class AgentCommunicationProtocol:
    def __init__(self, agents: List['SpecializedAgent']):
        self.agents = {agent.name: agent for agent in agents}
        self.communication_log = []

    def broadcast_message(self, sender: str, message: str, recipients: Optional[List[str]] = None):
        """
        Broadcast a message from one agent to others.
        If recipients are not specified, message goes to all agents except the sender.
        """
        if not recipients:
            recipients = [name for name in self.agents.keys() if name != sender]

        for recipient_name in recipients:
            if recipient_name in self.agents:
                response = self.agents[recipient_name].receive_message(sender, message)
                self.communication_log.append({
                    'sender': sender,
                    'recipient': recipient_name,
                    'message': message,
                    'response': response
                })

    def get_communication_history(self):
        """Retrieve the communication log."""
        return self.communication_log


class SpecializedAgent:
    def __init__(self, name: str, domain: str, config: RAGConfig):
        """
        Initialize a specialized agent with a specific domain expertise.

        :param name: Unique name of the agent
        :param domain: Specialized domain of knowledge
        :param config: RAG configuration
        """
        self.name = name
        self.domain = domain
        self.config = config
        self.embedding_model = SentenceTransformerEmbedding(config)
        self.vector_store = VectorStore(config, self.embedding_model)
        self.llm = LocalLLM(config)
        self.communication_protocol = None

    def set_communication_protocol(self, protocol: AgentCommunicationProtocol):
        """Set the communication protocol for the agent."""
        self.communication_protocol = protocol

    def ingest_documents(self, directory_path: str):
        """Ingest documents specific to the agent's domain."""
        document_processor = DocumentProcessor(self.config)
        documents = document_processor.load_documents(directory_path)
        chunks = document_processor.process_documents(documents)
        self.vector_store.create_vector_store(chunks)

    def query(self, query: str) -> str:
        """
        Process a query within the agent's domain,
        with optional inter-agent communication.
        """
        # If no vector store is loaded, try to load from path
        if not self.vector_store.vector_store:
            if not self.vector_store.load_vector_store():
                return f"{self.name}: No documents have been ingested in my domain."

        # Retrieve relevant documents
        retrieved_docs = self.vector_store.similarity_search(query)

        # Prepare system message with domain context
        system_message = f"""You are the {self.name} agent, specialized in {self.domain} domain.
        Your task is to provide accurate, relevant information based on the query and the retrieved context.
        If you lack sufficient information, you may request help from other specialized agents."""

        context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

        # First attempt to answer
        initial_response_prompt = f"""Query: {query}

        Context:
        {context}

        Based on the context provided, answer the query from the perspective of a {self.domain} expert.
        Identify if you need additional expertise to fully address the query.
        """

        initial_response = self.llm.generate(initial_response_prompt, system_message)

        # Check if inter-agent communication is needed
        need_help_prompt = f"""Domain: {self.domain}
        Query: {query}
        Current Response: {initial_response}

        Determine if this response requires expertise from other domains.
        If additional domain expertise is needed, specify which domains would help.
        Respond with a comma-separated list of domain names, or "NONE" if no help is needed.
        """

        needed_domains = self.llm.generate(need_help_prompt, system_message).strip()

        # Inter-agent communication
        if needed_domains.upper() != "NONE" and self.communication_protocol:
            needed_domains_list = [domain.strip() for domain in needed_domains.split(',')]

            # Broadcast message to relevant agents
            request_recipients = [
                agent_name for agent_name, agent in self.communication_protocol.agents.items()
                if agent.domain in needed_domains_list and agent_name != self.name
            ]

            if request_recipients:
                self.communication_protocol.broadcast_message(
                    sender=self.name,
                    message=f"Need help with query: {query}. Current context: {initial_response}",
                    recipients=request_recipients
                )

        return initial_response

    def receive_message(self, sender: str, message: str) -> str:
        """
        Handle messages from other agents.

        :param sender: Name of the agent sending the message
        :param message: Content of the message
        :return: Response to the message
        """
        system_message = f"""You are the {self.name} agent, receiving a message from {sender}.
        Analyze the message and provide a helpful response using your domain expertise."""

        response_prompt = f"""Message from {sender}:
        {message}

        Provide a helpful response that leverages your {self.domain} expertise.
        """

        return self.llm.generate(response_prompt, system_message)


# Multi-Agent RAG System
class MultiAgentRAG:
    def __init__(self, config: RAGConfig):
        """
        Initialize a multi-agent RAG system with predefined specialized agents.

        :param config: RAG configuration
        """
        self.config = config

        # Create specialized agents
        self.agents = [
            SpecializedAgent("TechAgent", "technology and computer science", config),
            SpecializedAgent("ScientificAgent", "scientific research", config),
            SpecializedAgent("BusinessAgent", "business and economics", config),
            SpecializedAgent("HistoryAgent", "history and humanities", config)
        ]

        # Set up communication protocol
        self.communication_protocol = AgentCommunicationProtocol(self.agents)

        # Configure communication for each agent
        for agent in self.agents:
            agent.set_communication_protocol(self.communication_protocol)

    def ingest_documents(self, agent_name: str, directory_path: str):
        """
        Ingest documents for a specific agent.

        :param agent_name: Name of the agent
        :param directory_path: Path to documents
        """
        agent = next((agent for agent in self.agents if agent.name == agent_name), None)
        if agent:
            agent.ingest_documents(directory_path)
        else:
            print(f"Agent {agent_name} not found.")

    def query(self, query: str) -> Dict[str, str]:
        """
        Process a query across all agents.

        :param query: User query
        :return: Responses from different agents
        """
        responses = {}
        for agent in self.agents:
            response = agent.query(query)
            responses[agent.name] = response

        return responses

    def get_communication_history(self):
        """
        Retrieve the communication history between agents.

        :return: List of communication logs
        """
        return self.communication_protocol.get_communication_history()


def main():
    config = RAGConfig()
    multi_agent_rag = MultiAgentRAG(config)

    while True:
        print("\nMulti-Agent RAG System")
        print("1. Ingest documents for an agent")
        print("2. Ask a cross-domain question")
        print("3. View communication history")
        print("4. Exit")

        choice = input("Enter your choice (1-4): ")

        if choice == "1":
            agent_name = input("Enter agent name (TechAgent, ScientificAgent, BusinessAgent, HistoryAgent): ")
            directory = input("Enter the directory path containing documents: ")
            multi_agent_rag.ingest_documents(agent_name, directory)

        elif choice == "2":
            query = input("Enter your cross-domain question: ")
            responses = multi_agent_rag.query(query)

            print("\nResponses from different agents:")
            for agent, response in responses.items():
                print(f"{agent}:\n{response}\n")

        elif choice == "3":
            history = multi_agent_rag.get_communication_history()
            print("\nCommunication History:")
            for log in history:
                print(f"Sender: {log['sender']}, Recipient: {log['recipient']}")
                print(f"Message: {log['message']}")
                print(f"Response: {log['response']}\n")

        elif choice == "4":
            print("Thank you for using Multi-Agent RAG. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()

Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu
Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu
Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu
Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu

Multi-Agent RAG System
1. Ingest documents for an agent
2. Ask a cross-domain question
3. View communication history
4. Exit
Enter your choice (1-4): 1
Enter agent name (TechAgent, ScientificAgent, BusinessAgent, Hi

KeyboardInterrupt: Interrupted by user

In [11]:
import os
import re
import time
import json
import uuid
from typing import List, Dict, Any, Optional, Tuple
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    TextLoader,
    PDFMinerLoader,
    UnstructuredMarkdownLoader,
    CSVLoader
)
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.schema import Document
from langchain.embeddings.base import Embeddings


class RAGConfig:
    def __init__(self):
        self.embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
        self.llm_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.torch_dtype = torch.float16 if self.device == "cuda" else torch.float32
        self.vector_db_path = "vector_db"
        self.chunk_size = 1000
        self.chunk_overlap = 200
        self.top_k = 5
        self.similarity_threshold = 0.7
        self.max_iterations = 3
        self.thinking_steps = True
        self.learning_rate = 0.1  # Dynamic learning rate


class KnowledgeGraph:
    """
    Advanced knowledge representation system that tracks
    relationships between concepts, confidence levels,
    and sources of information.
    """
    def __init__(self):
        self.nodes = {}  # Concept nodes
        self.edges = {}  # Relationships between concepts

    def add_concept(self, concept: str, initial_confidence: float = 0.5):
        """Add a new concept to the knowledge graph."""
        if concept not in self.nodes:
            self.nodes[concept] = {
                'confidence': initial_confidence,
                'sources': set(),
                'related_concepts': set()
            }

    def add_relationship(self, concept1: str, concept2: str, relationship_type: str):
        """
        Add a relationship between two concepts.
        Relationship types could be: 'is_a', 'part_of', 'related_to', etc.
        """
        if concept1 not in self.nodes:
            self.add_concept(concept1)
        if concept2 not in self.nodes:
            self.add_concept(concept2)

        if concept1 not in self.edges:
            self.edges[concept1] = {}

        self.edges[concept1][concept2] = {
            'type': relationship_type,
            'confidence': 0.5
        }

        # Bidirectional tracking of related concepts
        self.nodes[concept1]['related_concepts'].add(concept2)
        self.nodes[concept2]['related_concepts'].add(concept1)

    def update_confidence(self, concept: str, new_info_confidence: float, source: str):
        """
        Update the confidence of a concept based on new information.
        Uses a weighted learning approach.
        """
        if concept not in self.nodes:
            self.add_concept(concept)

        node = self.nodes[concept]
        node['sources'].add(source)

        # Weighted average of existing confidence and new information
        node['confidence'] = (
            (1 - self.config.learning_rate) * node['confidence'] +
            self.config.learning_rate * new_info_confidence
        )

    def get_concept_confidence(self, concept: str) -> float:
        """Retrieve the confidence level of a concept."""
        return self.nodes.get(concept, {}).get('confidence', 0)

    def find_related_concepts(self, concept: str, max_depth: int = 2) -> List[str]:
        """
        Find concepts related to the given concept within a specified depth.
        Uses a breadth-first search approach.
        """
        related = set()
        visited = set()
        queue = [(concept, 0)]

        while queue:
            current_concept, depth = queue.pop(0)

            if depth > max_depth:
                break

            if current_concept in visited:
                continue

            visited.add(current_concept)

            if current_concept != concept:
                related.add(current_concept)

            # Add neighboring concepts to the queue
            if current_concept in self.nodes:
                for neighbor in self.nodes[current_concept]['related_concepts']:
                    if neighbor not in visited and depth < max_depth:
                        queue.append((neighbor, depth + 1))

        return list(related)


class AdvancedSpecializedAgent:
    def __init__(self, name: str, domain: str, config: RAGConfig):
        """
        Enhanced agent with dynamic learning capabilities.
        """
        self.id = str(uuid.uuid4())  # Unique identifier for the agent
        self.name = name
        self.domain = domain
        self.config = config

        # Core components
        self.embedding_model = SentenceTransformerEmbedding(config)
        self.vector_store = VectorStore(config, self.embedding_model)
        self.llm = LocalLLM(config)

        # Advanced reasoning systems
        self.knowledge_graph = KnowledgeGraph()
        self.reasoning_cache = {}  # Cache for complex reasoning results

        # Learning and meta-cognitive tracking
        self.learning_history = []
        self.confidence_threshold = 0.7  # Confidence level for definitive answers

        # Inter-agent communication protocol
        self.communication_protocol = None

    def learn_from_interaction(self, query: str, response: str, source_agent: str):
        """
        Advanced learning mechanism that extracts and integrates knowledge.

        This method:
        1. Extracts key concepts from the interaction
        2. Updates the knowledge graph
        3. Tracks learning history
        """
        # Extract key concepts using LLM
        concept_extraction_prompt = f"""
        From the following query and response, extract:
        1. Key concepts
        2. Relationships between concepts
        3. Confidence level of information

        Query: {query}
        Response: {response}

        Return JSON format:
        {{
            "concepts": [
                {{"name": "concept_name", "confidence": 0.8}}
            ],
            "relationships": [
                {{"concept1": "A", "concept2": "B", "type": "is_a"}}
            ]
        }}
        """

        try:
            concept_json = json.loads(self.llm.generate(concept_extraction_prompt))

            # Update knowledge graph
            for concept in concept_json.get('concepts', []):
                self.knowledge_graph.update_confidence(
                    concept['name'],
                    concept['confidence'],
                    source_agent
                )

            # Add relationships
            for rel in concept_json.get('relationships', []):
                self.knowledge_graph.add_relationship(
                    rel['concept1'],
                    rel['concept2'],
                    rel['type']
                )

            # Track learning event
            self.learning_history.append({
                'timestamp': time.time(),
                'query': query,
                'source_agent': source_agent,
                'concepts_learned': [c['name'] for c in concept_json.get('concepts', [])]
            })

        except Exception as e:
            print(f"Learning extraction error: {e}")

    def generate_complex_reasoning(self, query: str) -> str:
        """
        Multi-step reasoning process that leverages the knowledge graph.

        Steps:
        1. Identify core concepts in the query
        2. Explore related concepts in the knowledge graph
        3. Generate a comprehensive reasoning path
        """
        # Concept identification
        concept_prompt = f"""
        Identify the core concepts in this query that require reasoning:
        Query: {query}

        Return a JSON list of key concepts.
        """

        try:
            core_concepts = json.loads(self.llm.generate(concept_prompt))

            # Explore related concepts
            related_concepts = []
            for concept in core_concepts:
                related_concepts.extend(
                    self.knowledge_graph.find_related_concepts(concept)
                )

            # Comprehensive reasoning
            reasoning_prompt = f"""
            Reasoning Task:
            Query: {query}

            Core Concepts: {core_concepts}
            Related Concepts: {related_concepts}

            Provide a step-by-step reasoning process that:
            1. Addresses the core query
            2. Leverages relationships between concepts
            3. Provides a nuanced, multi-perspective analysis
            """

            reasoning_response = self.llm.generate(reasoning_prompt)

            # Cache the reasoning for potential future use
            cache_key = hash(query)
            self.reasoning_cache[cache_key] = {
                'response': reasoning_response,
                'timestamp': time.time()
            }

            return reasoning_response

        except Exception as e:
            print(f"Complex reasoning error: {e}")
            return "Unable to generate complex reasoning."

    def query(self, query: str) -> Dict[str, Any]:
        """
        Enhanced query method with multi-stage reasoning and learning.
        """
        # Attempt to use cached reasoning first
        cache_key = hash(query)
        if cache_key in self.reasoning_cache:
            cached_result = self.reasoning_cache[cache_key]
            if time.time() - cached_result['timestamp'] < 3600:  # 1-hour cache
                return {
                    'response': cached_result['response'],
                    'source': 'reasoning_cache',
                    'confidence': 0.9
                }

        # Vector store retrieval
        retrieved_docs = self.vector_store.similarity_search(query)

        # Complex reasoning generation
        reasoning_response = self.generate_complex_reasoning(query)

        # Prepare final response
        final_response = {
            'response': reasoning_response,
            'retrieved_docs': [doc.page_content for doc in retrieved_docs],
            'source': self.name,
            'confidence': self.knowledge_graph.get_concept_confidence(query)
        }

        return final_response


class AdvancedMultiAgentRAG:
    def __init__(self, config: RAGConfig):
        """
        Advanced multi-agent system with dynamic learning and reasoning.
        """
        self.config = config
        self.agents = [
            AdvancedSpecializedAgent("SyntheticIntelligenceAgent", "AI and cognitive systems", config),
            AdvancedSpecializedAgent("GlobalSystemsAgent", "complex interdisciplinary systems", config),
            AdvancedSpecializedAgent("EmergentTechnologiesAgent", "cutting-edge technological innovations", config)
        ]

        # Collaborative learning environment
        self.collaborative_knowledge_graph = KnowledgeGraph()

    def cross_agent_learning(self, query: str, responses: Dict[str, Dict]):
        """
        Facilitate learning across agents by sharing knowledge.
        """
        for agent_name, response in responses.items():
            for other_agent in self.agents:
                if other_agent.name != agent_name:
                    other_agent.learn_from_interaction(
                        query,
                        response['response'],
                        agent_name
                    )

    def query(self, query: str) -> Dict[str, Dict]:
        """
        Advanced query method with collaborative reasoning.
        """
        responses = {}

        # Parallel query processing
        for agent in self.agents:
            responses[agent.name] = agent.query(query)

        # Cross-agent learning
        self.cross_agent_learning(query, responses)

        return responses


def main():
    config = RAGConfig()
    multi_agent_rag = AdvancedMultiAgentRAG(config)

    while True:
        print("\nAdvanced Multi-Agent RAG System")
        print("1. Ask a complex query")
        print("2. View agent learning histories")
        print("3. Exit")

        choice = input("Enter your choice (1-3): ")

        if choice == "1":
            query = input("Enter a complex, multi-domain question: ")
            responses = multi_agent_rag.query(query)

            print("\nResponses from different agents:")
            for agent_name, response in responses.items():
                print(f"\n{agent_name}:")
                print(f"Response: {response['response']}")
                print(f"Confidence: {response['confidence']}")

        elif choice == "2":
            for agent in multi_agent_rag.agents:
                print(f"\n{agent.name} Learning History:")
                for event in agent.learning_history:
                    print(f"Timestamp: {time.ctime(event['timestamp'])}")
                    print(f"Query: {event['query']}")
                    print(f"Source Agent: {event['source_agent']}")
                    print(f"Concepts Learned: {event['concepts_learned']}\n")

        elif choice == "3":
            print("Thank you for using Advanced Multi-Agent RAG. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()

Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu
Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu
Loading embedding model sentence-transformers/all-mpnet-base-v2...
Embedding model loaded successfully
Loading LLM model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
LLM model loaded successfully on cpu

Advanced Multi-Agent RAG System
1. Ask a complex query
2. View agent learning histories
3. Exit
Enter your choice (1-3): 1
Enter a complex, multi-domain question: e


ValueError: Vector store not initialized. Please create or load a vector store first.

In [2]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00


In [3]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 8.0 MB/s eta 

In [6]:
import os
import uuid
import asyncio
import json
from typing import List, Dict, Any, Optional

import chromadb
from chromadb.config import Settings
from chromadb.api.types import Embedding, Documents, IDs

from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (
    DirectoryLoader,
    TextLoader,
    PDFLoader,
    CSVLoader,
    UnstructuredMarkdownLoader
)
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnableConfig
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

class ChromaDBManager:
    def __init__(self, persist_directory: str = "./chroma_db"):
        """
        Initialize ChromaDB client with custom configuration
        """
        # Ensure directory exists
        os.makedirs(persist_directory, exist_ok=True)

        # ChromaDB client configuration
        self.client = chromadb.PersistentClient(path=persist_directory)

        # Collection cache
        self._collections = {}

    def create_collection(
        self,
        name: str,
        embedding_function=None,
        metadata: Optional[Dict] = None
    ):
        """
        Create or get a ChromaDB collection

        :param name: Unique name for the collection
        :param embedding_function: Custom embedding function
        :param metadata: Optional metadata for the collection
        :return: ChromaDB collection
        """
        try:
            # Check if collection already exists
            if name in self._collections:
                return self._collections[name]

            # Create or get collection
            collection = self.client.create_collection(
                name=name,
                embedding_function=embedding_function,
                metadata=metadata or {}
            )

            # Cache the collection
            self._collections[name] = collection
            return collection

        except Exception as e:
            print(f"Error creating collection {name}: {e}")
            return None

    def add_documents(
        self,
        collection_name: str,
        documents: Documents,
        embeddings: List[Embedding],
        ids: IDs
    ):
        """
        Add documents to a ChromaDB collection

        :param collection_name: Name of the collection
        :param documents: List of document texts
        :param embeddings: List of embeddings
        :param ids: List of unique document IDs
        """
        collection = self._collections.get(collection_name)
        if not collection:
            raise ValueError(f"Collection {collection_name} not found")

        collection.add(
            documents=documents,
            embeddings=embeddings,
            ids=ids
        )

    def query(
        self,
        collection_name: str,
        query_embeddings: List[Embedding],
        n_results: int = 5
    ):
        """
        Query a ChromaDB collection

        :param collection_name: Name of the collection
        :param query_embeddings: Embedding of the query
        :param n_results: Number of results to return
        :return: Query results
        """
        collection = self._collections.get(collection_name)
        if not collection:
            raise ValueError(f"Collection {collection_name} not found")

        return collection.query(
            query_embeddings=query_embeddings,
            n_results=n_results
        )

class MultiAgentRAGConfig:
    def __init__(self):
        # Embedding and model configurations
        self.embedding_model = "sentence-transformers/all-mpnet-base-v2"
        self.llm_model = "gpt-3.5-turbo"

        # Directory configurations
        self.document_dir = "./documents"
        self.chroma_dir = "./chroma_db"

        # Processing parameters
        self.chunk_size = 1000
        self.chunk_overlap = 200
        self.top_k_results = 5

class SpecializedAgent:
    def __init__(
        self,
        name: str,
        domain: str,
        config: MultiAgentRAGConfig,
        chroma_manager: ChromaDBManager
    ):
        self.id = str(uuid.uuid4())
        self.name = name
        self.domain = domain
        self.config = config
        self.chroma_manager = chroma_manager

        # Embedding setup
        self.embeddings = HuggingFaceEmbeddings(
            model_name=config.embedding_model
        )

        # Language model
        self.llm = ChatOpenAI(
            model=config.llm_model,
            temperature=0.7
        )

        # Create a collection for this agent
        self.collection = self.chroma_manager.create_collection(
            name=f"{name}_collection",
            embedding_function=self.embeddings.embed_documents
        )

    def ingest_documents(self, document_path: str):
        """
        Ingest and process documents for the agent's domain
        """
        # Document loaders for different file types
        loader_mapping = {
            "*.txt": TextLoader,
            "*.pdf": PDFLoader,
            "*.csv": CSVLoader,
            "*.md": UnstructuredMarkdownLoader
        }

        # Text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.config.chunk_size,
            chunk_overlap=self.config.chunk_overlap
        )

        # Collect and process documents
        all_documents = []
        for pattern, loader_cls in loader_mapping.items():
            try:
                loader = DirectoryLoader(
                    document_path,
                    glob=pattern,
                    loader_cls=loader_cls
                )
                docs = loader.load()
                all_documents.extend(docs)
            except Exception as e:
                print(f"Error loading {pattern} documents: {e}")

        # Split documents
        splits = text_splitter.split_documents(all_documents)

        # Prepare for ChromaDB ingestion
        documents = [doc.page_content for doc in splits]
        embeddings = self.embeddings.embed_documents(documents)
        ids = [str(uuid.uuid4()) for _ in documents]

        # Add to ChromaDB collection
        self.chroma_manager.add_documents(
            collection_name=f"{self.name}_collection",
            documents=documents,
            embeddings=embeddings,
            ids=ids
        )

        return len(documents)

    async def query(self, query: str):
        """
        Query the agent's knowledge base
        """
        # Embed the query
        query_embedding = self.embeddings.embed_query(query)

        # Perform ChromaDB query
        results = self.chroma_manager.query(
            collection_name=f"{self.name}_collection",
            query_embeddings=[query_embedding],
            n_results=self.config.top_k_results
        )

        # Prepare context
        context = "\n\n".join(results['documents'][0])

        # Generate response using LLM
        response_prompt = PromptTemplate.from_template(
            """You are a {domain} expert.
            Context: {context}

            Question: {query}

            Provide a comprehensive and insightful answer based on the context.
            """
        )

        # Create response chain
        response_chain = (
            response_prompt
            | self.llm
            | StrOutputParser()
        )

        # Generate response
        response = await response_chain.ainvoke({
            "domain": self.domain,
            "context": context,
            "query": query
        })

        return {
            "agent": self.name,
            "response": response,
            "retrieved_docs": results['documents'][0]
        }

class MultiAgentRAGSystem:
    def __init__(self, config: MultiAgentRAGConfig):
        # ChromaDB Manager
        self.chroma_manager = ChromaDBManager(
            persist_directory=config.chroma_dir
        )

        # Create specialized agents
        self.agents = [
            SpecializedAgent(
                "TechInnovationAgent",
                "technology and innovation",
                config,
                self.chroma_manager
            ),
            SpecializedAgent(
                "ScientificResearchAgent",
                "scientific research",
                config,
                self.chroma_manager
            ),
            SpecializedAgent(
                "GlobalSystemsAgent",
                "complex global systems",
                config,
                self.chroma_manager
            )
        ]

    def ingest_documents(self):
        """
        Ingest documents for all agents
        """
        results = {}
        for agent in self.agents:
            doc_count = agent.ingest_documents(config.document_dir)
            results[agent.name] = doc_count
        return results

    async def cross_domain_query(self, query: str):
        """
        Execute query across multiple agents asynchronously
        """
        # Parallel query execution
        agent_responses = await asyncio.gather(
            *[agent.query(query) for agent in self.agents]
        )

        # Synthesize responses using the first agent's LLM
        synthesizer = self.agents[0].llm
        synthesis_prompt = PromptTemplate.from_template(
            """Synthesize responses from multiple domain experts:

            Query: {query}

            Agent Responses:
            {responses}

            Provide a comprehensive, multi-perspective answer
            that integrates insights from different domains.
            """
        )

        # Synthesize response
        synthesis_chain = (
            synthesis_prompt
            | synthesizer
            | StrOutputParser()
        )

        synthesized_response = await synthesis_chain.ainvoke({
            "query": query,
            "responses": str(agent_responses)
        })

        return {
            "agent_responses": agent_responses,
            "synthesized_response": synthesized_response
        }

async def main():
    # Configuration
    config = MultiAgentRAGConfig()

    # Initialize RAG System
    rag_system = MultiAgentRAGSystem(config)

    while True:
        print("\nChroma Multi-Agent RAG System")
        print("1. Ingest Documents")
        print("2. Ask a Cross-Domain Question")
        print("3. Exit")

        choice = input("Enter your choice (1-3): ")

        if choice == "1":
            # Ingest documents
            ingest_results = rag_system.ingest_documents()
            print("Document Ingestion Results:")
            for agent, count in ingest_results.items():
                print(f"{agent}: {count} documents processed")

        elif choice == "2":
            # Cross-domain query
            query = input("Enter your complex question: ")

            result = await rag_system.cross_domain_query(query)

            print("\nAgent Responses:")
            for response in result['agent_responses']:
                print(f"{response['agent']} says:")
                print(response['response'])
                print("Retrieved Documents:")
                for doc in response['retrieved_docs']:
                    print(f"- {doc[:200]}...")
                print("\n")

            print("\nSynthesized Response:")
            print(result['synthesized_response'])

        elif choice == "3":
            print("Goodbye!")
            break

if __name__ == "__main__":
    asyncio.run(main())

ImportError: cannot import name 'PDFLoader' from 'langchain.document_loaders' (/usr/local/lib/python3.11/dist-packages/langchain/document_loaders/__init__.py)

In [ ]:
import os
import json
import gradio as gr
from typing import List, Dict, Any, Tuple
import torch
from threading import Thread
from datetime import datetime
import time

# Import the RAG system components
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

# Import your RAG system classes
# Assuming the original code is in a file called rag_system.py


# Global variables
rag_system = None
config = None
is_initialized = False
document_status = "No documents ingested yet"
model_status = "System not initialized"


def initialize_system():
    """Initialize the RAG system in the background"""
    global rag_system, config, is_initialized, model_status

    try:
        # Update status
        model_status = "Initializing models... This may take a few minutes"

        # Initialize configuration
        config = RAGConfig()

        # Initialize the system
        rag_system = AgenticRAG(config)

        # Update status
        model_status = f"System initialized successfully. Using device: {config.device}"
        is_initialized = True

        # Try to load existing vector store
        if os.path.exists(config.vector_db_path):
            rag_system.vector_store.load_vector_store()
            global document_status
            document_status = "Previously ingested documents loaded"

    except Exception as e:
        model_status = f"Error initializing system: {str(e)}"
        print(f"Initialization error: {e}")


def init_background():
    """Start initialization in background thread"""
    thread = Thread(target=initialize_system)
    thread.daemon = True
    thread.start()
    return "Initializing system in background. Please wait..."


def ingest_documents(directory_path):
    """Ingest documents from the specified directory"""
    global rag_system, document_status

    if not is_initialized:
        return "System not initialized yet. Please wait."

    if not os.path.exists(directory_path):
        return f"Directory not found: {directory_path}"

    try:
        # Update status
        document_status = f"Ingesting documents from {directory_path}..."

        # Ingest documents
        rag_system.ingest_documents(directory_path)

        # Update status
        document_status = f"Successfully ingested documents from {directory_path}"
        return f"Successfully ingested documents from {directory_path}"

    except Exception as e:
        error_msg = f"Error ingesting documents: {str(e)}"
        document_status = error_msg
        return error_msg


def query_system(query, thinking_steps=True, max_iterations=3):
    """Query the RAG system with the given question"""
    global rag_system, config, is_initialized, document_status

    if not is_initialized:
        return "System not initialized yet. Please wait."

    if not rag_system.vector_store.vector_store:
        try:
            if not rag_system.vector_store.load_vector_store():
                return "No documents have been ingested. Please ingest documents first."
        except Exception as e:
            return f"Error loading vector store: {str(e)}"

    try:
        # Update configuration
        config.thinking_steps = thinking_steps
        config.max_iterations = max_iterations

        # Process query
        start_time = time.time()
        response = rag_system.query(query)
        end_time = time.time()

        # Return formatted response
        time_taken = end_time - start_time
        return f"Response (took {time_taken:.2f} seconds):\n\n{response}"

    except Exception as e:
        return f"Error processing query: {str(e)}"


def get_system_info():
    """Get information about the system configuration"""
    global config, is_initialized, document_status, model_status

    if not is_initialized:
        return model_status

    info = {
        "Status": model_status,
        "Device": config.device,
        "Embedding Model": config.embedding_model_name,
        "LLM Model": config.llm_model_name,
        "Document Status": document_status,
        "Vector DB Path": config.vector_db_path,
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return "\n".join([f"{k}: {v}" for k, v in info.items()])


# Create Gradio interface
def create_ui():
    with gr.Blocks(title="AgenticRAG System", theme=gr.themes.Soft()) as app:
        gr.Markdown("# AgenticRAG System with Local LLM")
        gr.Markdown("A RAG system with local models for document ingestion and querying.")

        with gr.Tab("System Status"):
            init_button = gr.Button("Initialize System", variant="primary")
            status_output = gr.Textbox(label="System Status", value="System not initialized", lines=10)
            refresh_button = gr.Button("Refresh Status")

            init_button.click(init_background, outputs=status_output)
            refresh_button.click(get_system_info, outputs=status_output)

        with gr.Tab("Ingest Documents"):
            gr.Markdown("### Ingest Documents")
            dir_input = gr.Textbox(label="Document Directory Path", placeholder="/path/to/documents")
            ingest_button = gr.Button("Ingest Documents", variant="primary")
            ingest_output = gr.Textbox(label="Ingestion Status", lines=5)

            ingest_button.click(ingest_documents, inputs=dir_input, outputs=ingest_output)

        with gr.Tab("Ask Questions"):
            gr.Markdown("### Query the System")

            with gr.Row():
                with gr.Column():
                    query_input = gr.Textbox(label="Your Question", placeholder="What would you like to know?", lines=3)
                    with gr.Row():
                        thinking_checkbox = gr.Checkbox(label="Enable Thinking Steps", value=True)
                        iterations_slider = gr.Slider(minimum=1, maximum=5, value=3, step=1, label="Max Iterations")

                    query_button = gr.Button("Submit Query", variant="primary")

            response_output = gr.Textbox(label="Response", lines=15)

            query_button.click(
                query_system,
                inputs=[query_input, thinking_checkbox, iterations_slider],
                outputs=response_output
            )

        # Auto-initialize on startup
        app.load(fn=init_background, outputs=status_output)

    return app

# Launch the app
if __name__ == "__main__":
    ui = create_ui()
    ui.launch(share=True, server_name="0.0.0.0", server_port=7861)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c08bd68e232ea1e97d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
